In [ ]:
# django-admin startproject mytodo .
# python manage.py startapp todo

# mytodo/settings.py 
INSTALLED_APPS = [
    "django.contrib.admin",
    "django.contrib.auth",
    "django.contrib.contenttypes",
    "django.contrib.sessions",
    "django.contrib.messages",
    "django.contrib.staticfiles",
    "rest_framework", # 추가
    "todo" # 추가
]

TIME_ZONE = "Asia/Seoul"

In [ ]:
# python manage.py make migrations
# python manage.py migrate
# python manage.py createsuperuser

In [ ]:
# todo/models.py
from django.db import models

# Create your models here.
class Todo(models.Model):
    title = models.CharField(max_length = 100)
    description = models.TextField(blank = True)
    created = models.DateTimeField(auto_now_add = True)
    complete = models.BooleanField(default = False)
    important = models.BooleanField(default = False)

    def __str__(self):
        return self.title

- todo 전체 조회 api 만들기

    - 전체 조회용 시리얼라이저
        - 제목, 완료 여부, 중요 여부만 포함한 간단한 시리얼라이저
        
    - 뷰
        - complete 가 False인 todo 들을 조회
        - get 방식으로 요청을 처리
        
    - urls 
        - 127.0.0.1:8000/todo/

In [ ]:
# todo/serializers.py
from rest_framework import serializers
from todo.models import Todo

class TodoSimpleSerializer(serializers.ModelSerializer):
    class Meta:
        model = Todo
        fields = ("id", "title", "complete", "important")

In [ ]:
# todo/vies.py
from rest_framework import status
from rest_framework.response import Response
from rest_framework.views import APIView
from todo.models import Todo
from todo.serializers import TodoSimpleSerializer

# Create your views here.
class TodosAPIView(APIView):
    def get(self, request):
        todos = Todo.objects.filter(complete = False)
        serializer = TodoSimpleSerializer(todos, many = True)
        return Response(serializer.data, status = status.HTTP_200_OK)

In [ ]:
# mytodo/urls.py

from django.urls import path, include

urlpatterns = [
    path("todo/", include("todo.urls")),
]

In [ ]:
# todo/urls.py
from django.urls import path
from todo import views

urlpatterns = [
    path("", views.TodosAPIView.as_view()),
    ]

- 상세 조회 기능 개발
    - 시리얼라이저
        - Todo 모델 id, 제목, 설명, 작성일자, 완료여부, 중요 여부 를 포함
        
    - 뷰
        - id를 입력받으면 해당 id의 Todo 데이터를 조회
        - get

In [ ]:
# todo/views.py
from todo.serializers import TodoSimpleSerializer, TodoDetailSerializer
from rest_framework.generics import get_object_or_404 

class TodoAPIView(APIView):
    def get(self, request, id):
        todo = get_object_or_404(Todo, id = id)
        serializer = TodoDetailSerializer(todo)
        return Response(serializer.data, status = status.HTTP_200_OK)

In [ ]:
# todo/urls.py
from django.urls import path
from todo import views

urlpatterns = [
    path("<int:id>/", views.TodoAPIView.as_view()),
    ]

- todo 생성 api 개발
    - 시리얼라이저
        - todo 생성에 필요한 입력값은 title과 description, important
        
    - 뷰
        - post 요청
        - post 요청으로 전달받은 데이터를 시리얼라이저에서 유효성 검사를 하고 todo 데이터 추가
        
    - url
        - (post) 127.0.0.1:8000/todo/

In [ ]:
# todo/serializers.py

class TodoCreateSerializer(serializers.ModelSerializer):
    class Meta:
        model = Todo
        fields = ("title", "description", "important")
        
# todo/views.py
from todo.serializers import TodoCreateSerializer

class TodosAPIView(APIView):
    def post(self, request):
        serializer = TodoCreateSerializer(data = request.data)

        if serializer.is_valid():
            serializer.save()
            return Response(serializer.data, status = status.HTTP_201_CREATED)
        
        return Response(serializer.errors, status = status.HTTP_400_BAD_REQUEST)
    

# todo/urls.py
from django.urls import path
from todo import views

urlpatterns = [
    path("", views.TodosAPIView.as_view()),
    ]

- 수정 & 완료 기능
    - 수정 기능
        - 시리얼라이저
            - 입력값은 title과 description, important
            
        - 뷰
            - id를 입력받고 id가 일치하는(수정할)데이터를 조회하고 조회되지 않으면 404 에러
            - 유효성 검사 후 유효하면 데이터 수정 / 유효하지 않으면 400 응답코드
            
        - urls
            - (put) 127.0.0.1:8000/todo/수정할 todo의 id/
            
    - 완료 목록 조회 기능
        - 시리얼라이저
            - id, title, complete, important 사용
            
        - 뷰
            - complete 가 True인 데이터를 조회
        
        - urls
            - (get) 127.0.0.1:8000/todo/done/
            
    - 완료 기능
        - 시리얼라이저
            - id, title, description, created, complete, import
            
        - 뷰
            - id를 입력받고 id가 일치하는(완료 할)데이터를 조회하고 조회되지 않으면 404 에러 
            - 조회된 데이터의 complete 속성 값을 True로 변환 후 저장
            
        - urls
            - (get) 127.0.0.1:8000/todo/done/완료할 todo의 id/